<a href="https://colab.research.google.com/github/flying-bear/infosearch/blob/master/3_vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

let us install lemmatization for fasttext model

In [1]:
!pip install pymorphy2

**IMPORTS**

In [276]:
%load_ext autoreload

import json
import numpy as np
import os
import pandas as pd
import pymorphy2
import re
import sys
import shutil
import tensorflow as tf


from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
from google.colab import drive
from math import log
from sklearn.feature_extraction.text import CountVectorizer
from time import time

trained_size = 2000

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
def enum_sort(arr): 
  """
  sorts list by values and returns sorted ids
  :param arr: list to be sorted
  :return: list of ids sorted by thir values in arr
  """
  return [x[0] for x in sorted(enumerate(arr), key=lambda x:x[1], reverse=True)]

oh, well, colab stuff <br>

(**I AM SO SORRY FOR ABSOLUTE PATHS BUT MY LAPTOP IS DEAD**)

In [42]:
os.path.abspath('.')

'/content'

In [57]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
root_path = 'gdrive/My Drive/studies/HSE/prog/infosearch/3 vec'

In [63]:
os.path.abspath('.')

'/content'

In [62]:
#shutil.copytree('gdrive/My Drive/studies/HSE/prog/infosearch/3 vec/elmo', 'content/elmo')
#shutil.copytree(root_path+'/simple_elmo/bilm', 'content/bilm')
#shutil.copy('gdrive/My Drive/studies/HSE/prog/infosearch/2 bm25/quora_question_pairs_rus.csv', 'content')
#shutil.copytree(root_path+'/fasttext', 'content/fasttext')

'content/elmo'

**let's load the dataset**

In [28]:
questions = pd.read_csv('content/quora_question_pairs_rus.csv', index_col=0).dropna()
questions.head()

,question1,question2,is_duplicate
0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1


**only some texts will be used, a part defined by trained_size constant above**

In [0]:
train_texts = questions[:trained_size]['question2'].dropna().tolist()

**let's define preprocessing and lemmatiztion for fasttext**

In [0]:
morph=pymorphy2.MorphAnalyzer()

def strip_split(text): 
  """
  cleans a text of punctuation and case
  
  :param text: string to be cleaned
  :return: a list of strings (words) in lowercase and stripped of punctuation
  """
  low = text.lower()
  stripped = re.sub('!|\.|,|#|$|%|\\|\'|\(|\)|-|\+|\*|/|\:|;|<|>|=|\?|\[|\]|@|^|_|`|{|}|~', '', low)
  words = stripped.split()
  return words

def lemmatize(text):
  """
  lemmatizes string using PyMorphy
  
  :param text: string to be lemmatized
  :return: list of lemmatized words (strings)
  """
  return [morph.parse(word)[0].normal_form for word in strip_split(text)]

## Задание

Реализуйте поиск по [Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian) на нескольких векторных моделях

    1. fasttext, модель ruscorpora_none_fasttextskipgram_300_2_2019
    2. elmo, модель ruwikiruscorpora_lemmas_elmo_1024_2019
    3. bert*, RuBERT - необязательно
   
Первые две обученные модели можно скачать на сайте [rusvectores](https://rusvectores.org/en/models/).

BERT делать необязательно, но если сделаете, 6 за курс у вас автоматом. Модель можно [найти тут](http://docs.deeppavlov.ai/en/master/features/models/bert.html).

In [0]:
def cos_sim(v1, v2):
  """
  cosine similarity of two vectors
  
  :param v1: np.array, first vector
  :param v2: np.array, second vector
  :return: float, cosine similarity of the vectors
  """
  return np.inner(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))

### __Задача 1__:    
Сравните время индексации корпуса для каждой модели 

**FASTTEXT**

load model

In [222]:
# если модель без тэгов
model_file = '/'.join(['content', 'fasttext', 'model.model'])
model = KeyedVectors.load(model_file)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


проверка наличия слова в словаре

In [223]:
lemmatize('Я полнейШий дебИл! Не умею ДУМАТЬ (головой)')

['я', 'полный', 'дебил', 'не', 'уметь', 'думать', 'голова']

In [224]:
print('я' in model.vocab, 'черепаха' in model.vocab)

False True


получение вектора слова

In [225]:
model['черепаха'][10:20]

array([-0.14401978, -0.34113285,  0.10848983, -0.41619024,  0.10033557,
        0.85966784, -0.13698106, -0.1241506 ,  0.06038719,  0.02368295],
      dtype=float32)

получение вектора документа

In [0]:
def doc_to_vec_fasttext(doc_text):
  """
  creates a vector out of a document using fasttext model
  
  :param doc_text: string to be transformed into vector
  :return: np.array, mean vector of word vectors in provided text
  """
  lemmas = lemmatize(doc_text) # сделали препроцессинг, получили леммы 
  # создаем маски для векторов 
  lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
  doc_vec = np.zeros((model.vector_size,))
  # если слово есть в модели, берем его вектор
  for idx, lemma in enumerate(lemmas):
    if lemma in model.vocab: # word in vocab
      try:
        lemmas_vectors[idx] = model[lemma]
      except AttributeError as e: # word in vocab but not in model
        print(e)
  if lemmas_vectors.shape[0] is not 0:  # проверка на случай, если на вход пришел пустой массив
    doc_vec = np.mean(lemmas_vectors, axis=0)
    return doc_vec

In [227]:
doc_to_vec_fasttext('я полны дебил то делать?')[10:20]

array([-0.09096584,  0.00550343, -0.039503  , -0.1670939 , -0.13090189,
       -0.03988818,  0.07535863,  0.04886302,  0.06259536, -0.12735136])

I see no point in writing a separate fasttext indexing function as it would be trivial (below)

In [228]:
print('fasttext')
start = time()
fasttext_doc2vec_corpus = []
for doc in train_texts:
  fasttext_doc2vec_corpus.append(doc_to_vec_fasttext(doc))
time_fasttext = time() - start
print(f'TIME: {str(time_fasttext)}')
print(f' example of text: {train_texts[0]},\n first 10 values from its vector:\n {fasttext_doc2vec_corpus[0][:10]}')

fasttext
TIME: 4.307932376861572
 example of text: что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад,
 first 10 values from its vector:
 [ 0.12532789  0.04991495  0.18384925  0.03815691 -0.00037019  0.0517497
  0.01790214 -0.11584419 -0.02457854  0.06156652]


**ELMO**

In [0]:
sys.path.append('/content/content')

model-specific imports

In [0]:
from elmo_helpers import tokenize, get_elmo_vectors, load_elmo_embeddings

tf.reset_default_graph()
elmo_path = 'content/elmo'

In [0]:
def crop_vec(vect, sent):
    """
    deletes empty vectors at the end of a sentence

    :param vect: np.array, vector from ELMo
    :param sent: list of str, tokenized sentence
    :return: np.array
    """
    cropped_vector = vect[:len(sent), :]
    cropped_vector = np.mean(cropped_vector, axis=0)
    return cropped_vector

In [232]:
tokenize('я ничего не могу сделать')

['я', 'ничего', 'не', 'могу', 'сделать']

In [233]:
batcher, sentence_character_ids, elmo_sentence_input = load_elmo_embeddings(elmo_path)

In [0]:
def doc_to_vec_elmo(doc_text):
  """ 
  creates a vector out of a document using fasttext model
  
  :param doc_text: string to be transformed into vector
  :param batcher, sentence_character_ids, elmo_sentence_input: ELMo model
  :return: np.array, mean vector of word vectors in provided text
    """
  tokens = [tokenize(doc_text)]
  with tf.Session() as sess:
      sess.run(tf.global_variables_initializer())
      vector = crop_vec(get_elmo_vectors(sess, tokens, batcher,
                                         sentence_character_ids,
                                         elmo_sentence_input)[0], tokens[0])
  return vector

In [235]:
doc_to_vec_elmo('тяжело очень мне')

Sentences in this batch: 1


array([-0.2806621 ,  0.22832812,  0.54048353, ..., -0.4438243 ,
       -0.13787274,  0.08398611], dtype=float32)

In [0]:
def elmo_indexing(tokenized, batcher, sentence_character_ids, elmo_sentence_input):
    """ 
    indexing texts using ELMo
    :param tokenized: list of lists of str, tokenized documents from the corpus
    :param batcher, sentence_character_ids, elmo_sentence_input: ELMo model

    :return: matrix of document vectors
    """
    with tf.Session() as sess:
        # It is necessary to initialize variables once before running inference.
        sess.run(tf.global_variables_initializer())
        indexed = []
        for i in range(200, len(tokenized)+1, 200):
            sentences = tokenized[i-200 : i]
            elmo_vectors = get_elmo_vectors(
                sess, sentences, batcher, sentence_character_ids, elmo_sentence_input)

            for vect, sent in zip(elmo_vectors, sentences):
                cropped_vector = crop_vec(vect, sent)
                indexed.append(cropped_vector)
    return indexed

In [237]:
start = time()
elmo_doc2vec_corpus = elmo_indexing([tokenize(q) for q in train_texts], batcher, sentence_character_ids, elmo_sentence_input)
print('elmo')
time_elmo = time() - start
print(f'TIME: {str(time_elmo)}')
print(f' example of text: {train_texts[0]},\n first 10 values from its vector:\n {elmo_doc2vec_corpus[0][:10]}')

Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200


elmo
TIME: 11.883128643035889
 example of text: что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад,
 first 10 values from its vector:
 [-0.26944843 -0.1237908   0.34158683  0.16239585 -0.635508    0.08462887
  0.3105996  -0.17464824  0.07552434  0.3336463 ]


**BM25 (lemmatized)**

In [0]:
k = 2.0
N = trained_size
vectorizer = CountVectorizer()

define mean text length

In [177]:
lens = [len(text.split()) for text in train_texts]
avgdl = sum(lens)/N
avgdl

9.5675

precompute a count matrix
<br> rows - documents
<br> columns - words

In [183]:
X = vectorizer.fit_transform([' '.join(lemmatize(q)) for q in train_texts])
count_matrix = X.toarray()
count_matrix.shape

(2000, 4489)

precompute tfs

In [0]:
tf_matrix = count_matrix / np.array(lens).reshape((-1, 1))

get a vocabulary that has the same indexation as the rows of the count matrix

In [0]:
vocabulary = vectorizer.get_feature_names()

get idfs

In [0]:
in_n_docs = np.count_nonzero(count_matrix, axis=0)

In [0]:
def IDF_modified(word):
    word_id = vocabulary.index(word)
    n = in_n_docs[word_id]
    return log((N - n + 0.5) / (n + 0.5))

In [0]:
idfs = [IDF_modified(word) for word in vocabulary]

compute bm25

In [0]:
def modify_tf(tf_value, doc_index, b=0.75):
    l = lens[doc_index]
    return (tf_value * (k + 1.0))/(tf_value + k * (1.0 - b + b * (l/avgdl)))

def modify_tf_matrix(tf_matrix, b=0.75): 
    enumed =  np.ndenumerate(tf_matrix)
    for i, tf_value in enumed:
        doc_index = i[0]
        tf_matrix[i] = modify_tf(tf_value, doc_index, b)
    return tf_matrix*idfs

In [194]:
start = time()
modified_tf_matrix = modify_tf_matrix(tf_matrix)
print('bm25')
time_bm25_lemm = time() - start
print(f'TIME Indexing: {str(time_bm25_lemm)}')

bm25
TIME Indexing: 12.373842000961304


In [0]:
def bm25_vector(query):
    vector = np.array(vectorizer.transform([' '.join(lemmatize(query))]).todense())[0]
    binary_vector = np.vectorize(lambda x: 1.0 if x != 0.0 else 0.0)(vector) ## neutralizes duplictes in the query (non-lineraity)
    return modified_tf_matrix.dot(binary_vector)

In [201]:
sum(bm25_vector('в воде'))

13.132411112541131

**indexation comparison**

In [203]:
print(f'let us compare indexation times on trained size {trained_size}:\n fasttext: {time_fasttext} sec\n elmo: {time_elmo} sec,\n  bm25: {time_bm25_lemm} sec ')

let us compare indexation times on trained size 2000:
 fasttext: 4.296413898468018 sec
 elmo: 11.95018744468689 sec,
  bm25: 12.373842000961304 sec 


### __Задача 2__:    
Выведите качество поиска для каждой модели +  BM25 для сравнения

Качество оцениваем так же, как в прошлом задании:
    - если в топ-5 результатов выдачи попал хоть один релевантный документ, выдача точная
    - если в топ-5 нет ни одного релеватного документа, выдача получает 0
   

In [0]:
def search_fasttext(query, n=trained_size): 
    """
    searches a given query in fasttext model, returns top n results, by default n = all found (the legth of collection)
    
    :param query: text to search for, string
    :param n: how many most relevant results to return, int
    :return: list of triplets (document id, document text, cos_sim to document text)
    """
    cos_sim_relevance = [cos_sim(doc_to_vec_fasttext(query), doc) for doc in fasttext_doc2vec_corpus]
    relevance_sorted_document_ids_top_n = enum_sort(cos_sim_relevance)[:n]
    return [(index, np.array(train_texts)[index], cos_sim_relevance[index]) for index in relevance_sorted_document_ids_top_n]

In [0]:
def search_elmo(query, n=trained_size): 
    """
    searches a given query in elmo model, returns top n results, by default n = all found (the legth of collection)
    
    :param query: text to search for, string
    :param n: how many most relevant results to return, int
    :return: list of triplets (document id, document text, cos_sim to document text)
    """
    query_vec = doc_to_vec_elmo(query)
    cos_sim_relevance = [cos_sim(query_vec, doc_vec) for doc_vec in elmo_doc2vec_corpus]
    relevance_sorted_document_ids_top_n = enum_sort(cos_sim_relevance)[:n]
    return [(index, np.array(train_texts)[index], cos_sim_relevance[index]) for index in relevance_sorted_document_ids_top_n]

In [0]:
def search_bm25(query, n=trained_size): 
    """
    searches a given query in bm25 model, returns top n results, by default n = all found (the legth of collection)
    
    :param query: text to search for, string
    :param n: how many most relevant results to return, int
    :return: list of triplets (document id, document text, cos_sim to document text)
    """
    bms = bm25_vector(query)
    relevance_sorted_document_ids_top_n = enum_sort(bms)[:n]
    return [(index, np.array(train_texts)[index], bms[index]) for index in relevance_sorted_document_ids_top_n]

In [0]:
query = 'Почему меня девушки не любят?'

In [248]:
search_fasttext(query, n=5)

[(705, 'почему я не хочу разговаривать с кем-то', 0.831970292342526),
 (644, 'как я могу быть геем и не люблю гей-секс', 0.8242458474426835),
 (1486,
  'как я могу сказать, действительно ли моя девушка любит меня?',
  0.8085746678015144),
 (1595,
  'почему я боюсь рассказать своему парню, что я его люблю',
  0.8055456291466281),
 (392,
  'почему люди ненавидят шарух хана и почему люди чувствуют, что он не индийский',
  0.7867779928670644)]

In [249]:
search_elmo(query, n=5)

Sentences in this batch: 1


[(1662,
  'что я должен делать, если моя подруга, которая обещала мне, что она никогда не покинет меня и не полюбит меня больше, чем моя мама оставила меня для кого-то другого',
  0.82276833),
 (1595, 'почему я боюсь рассказать своему парню, что я его люблю', 0.81353277),
 (1091, 'почему некоторые люди любят давать минеты', 0.7977652),
 (523, 'почему моя собака укусила меня', 0.79703826),
 (1486,
  'как я могу сказать, действительно ли моя девушка любит меня?',
  0.786887)]

In [250]:
search_bm25(query, n=5)

[(368, 'девушки любят черных парней без бороды', 7.573496882335712),
 (1101, 'почему сербы любят россиян', 6.689078862794888),
 (590, 'как девушка получает парня', 6.59057109708373),
 (257, 'почему не большинство индусов едят не-овощные', 3.9836102148059087),
 (418, 'почему не должен я смотреть порно', 3.9836102148059087)]

In [0]:
def test_q1_by_id(q1_index, model_name='fasttext'):
  """
  tests whether for a given question from quora question pairs relevant answer is in top five responses
  
  :param q1_index: int, question id from quora question pairs dataset
  :param model_name: string {'fasttext', 'elmo', 'bm25'}, 'fasttext' by default, which model to use for searching
  :return: 0 or 1, boolean accuracy
  """
  if model_name == 'fasttext':
    search = search_fasttext    
  elif model_name == 'elmo':  
    search = search_elmo
  elif model_name == 'bm25':
    search = search_bm25
  else:
    raise ValueError(f'model_name must be ether "fasttext", "elmo", or "bm25", but {model_name} was received')
  top_5_ids = [i for i, text, bm_25 in search(questions.iloc[q1_index]['question1'], n=5)]
  return 1.0 if q1_index in top_5_ids else 0.0

In [0]:
def test_over_multiple_questions(testsize=50, model_name='fasttext'):
  """
  tests search accuracy over a given number of questions form quora question pairs dataset
  
  :param testsize: int, 50 by default, number of questions to test over,
  :param model: string {'fasttext', 'elmo', 'bm25'}, 'fasttext' by default, which model to use for searching
  :return: float, mean accuracy over testsize number of questions
  """
  if model_name == 'fasttext':
    print('searching fasttext')   
  elif model_name == 'elmo':  
    print('searching elmo')
  elif model_name == 'bm25':
    print('searching bm25')
  testable = questions[(questions['is_duplicate'] == 1)&(questions.index < trained_size)]
  if testsize < len(testable):
      testable = testable[:testsize]
      print('testing on', testsize, 'questions')
  else:
      print('testing on', len(testable), 'questions')
  hit_count = 0.0
  for index in testable.index:
      hit_count += test_q1_by_id(index, model_name)
  return hit_count/len(testable.index)

**Let us comare accuracies**

no difference on 20 questions

In [279]:
n = 20
comparison_20 = []
for model_name in ['fasttext', 'elmo', 'bm25']:
  start = time()
  acc = test_over_multiple_questions(testsize=n, model_name=model_name)
  search_time = time() - start
  comparison_20.append((model_name, acc, search_time))

searching fasttext
testing on 20 questions
searching elmo
testing on 20 questions


Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1


searching bm25
testing on 20 questions


In [282]:
for model_name, accuracy, search_time in comparison_20:
  print(f'model: {model_name},\naccuracy: {accuracy},\nsearch time: {search_time}')

model: fasttext,
accuracy: 0.7,
search time: 84.62671709060669
model: elmo,
accuracy: 0.7,
search time: 44.6616473197937
model: bm25,
accuracy: 0.7,
search time: 0.47026586532592773


let us test 50, 100, 150

In [283]:
test_sized_comparison = {50 : [], 100 : [], 150 : []}
for n in test_sized_comparison:
  comparison = []
  for model_name in ['fasttext', 'elmo', 'bm25']:
    start = time()
    acc = test_over_multiple_questions(testsize=n, model_name=model_name)
    search_time = time() - start
    comparison.append((model_name, acc, search_time))
  test_sized_comparison[n] = comparison

searching fasttext
testing on 50 questions
searching elmo
testing on 50 questions


Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
S

searching bm25
testing on 50 questions
searching fasttext
testing on 100 questions
searching elmo
testing on 100 questions


Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
S

searching bm25
testing on 100 questions
searching fasttext
testing on 150 questions
searching elmo
testing on 150 questions


Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
S

searching bm25
testing on 150 questions


In [284]:
for n in test_sized_comparison:
  print('\n\n')
  print(f'test on {n} questions')
  for model_name, accuracy, search_time in comparison:
    print(f'model: {model_name},\naccuracy: {accuracy},\nsearch time: {search_time}')
    print()




test on 50 questions
model: fasttext,
accuracy: 0.7866666666666666,
search time: 578.7353513240814

model: elmo,
accuracy: 0.74,
search time: 336.6040380001068

model: bm25,
accuracy: 0.68,
search time: 3.2782135009765625




test on 100 questions
model: fasttext,
accuracy: 0.7866666666666666,
search time: 578.7353513240814

model: elmo,
accuracy: 0.74,
search time: 336.6040380001068

model: bm25,
accuracy: 0.68,
search time: 3.2782135009765625




test on 150 questions
model: fasttext,
accuracy: 0.7866666666666666,
search time: 578.7353513240814

model: elmo,
accuracy: 0.74,
search time: 336.6040380001068

model: bm25,
accuracy: 0.68,
search time: 3.2782135009765625

